In [35]:
import xml.etree.ElementTree as ET

#tree = ET.parse(r'C:\Users\Chris\Documents\projects\train-game\drawing.svg')
tree = ET.parse(r'drawing-nobg.svg')
root = tree.getroot()

root

ns = {'svg': 'http://www.w3.org/2000/svg',
      'ttr': 'https://train-game.github.io',
          'inkscape': 'http://www.inkscape.org/namespaces/inkscape'}

ET.register_namespace('', 'http://www.w3.org/2000/svg')
ET.register_namespace('inkscape', 'http://www.inkscape.org/namespaces/inkscape')
ET.register_namespace('sodipodi', 'http://sodipodi.sourceforge.net/DTD/sodipodi-0.dtd')


In [45]:
for node in root.iter():
    # Copy inkscape labels to element_ids
    label = node.attrib.get('{http://www.inkscape.org/namespaces/inkscape}label')
    if label:
        node.set('id', label)
        
    # Delete inkscape + sodipodi attributes on nodes
    keys = list(node.attrib.keys())
    for attrib_key in keys:
        if "inkscape" in attrib_key or "sodipodi" in attrib_key:
            node.attrib.pop(attrib_key)
        
    

node tag:  {http://www.w3.org/2000/svg}svg
node tag:  {http://sodipodi.sourceforge.net/DTD/sodipodi-0.dtd}namedview
node tag:  {http://www.w3.org/2000/svg}defs
node tag:  {http://www.inkscape.org/namespaces/inkscape}path-effect
node tag:  {http://www.inkscape.org/namespaces/inkscape}path-effect
node tag:  {http://www.w3.org/2000/svg}g
node tag:  {http://www.w3.org/2000/svg}circle
node tag:  {http://www.w3.org/2000/svg}circle
node tag:  {http://www.w3.org/2000/svg}circle
node tag:  {http://www.w3.org/2000/svg}circle
node tag:  {http://www.w3.org/2000/svg}circle
node tag:  {http://www.w3.org/2000/svg}circle
node tag:  {http://www.w3.org/2000/svg}circle
node tag:  {http://www.w3.org/2000/svg}circle
node tag:  {http://www.w3.org/2000/svg}circle
node tag:  {http://www.w3.org/2000/svg}circle
node tag:  {http://www.w3.org/2000/svg}circle
node tag:  {http://www.w3.org/2000/svg}circle
node tag:  {http://www.w3.org/2000/svg}circle
node tag:  {http://www.w3.org/2000/svg}circle
node tag:  {http://

In [44]:
print(dir(node))

['__bool__', '__class__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'attrib', 'clear', 'extend', 'find', 'findall', 'findtext', 'get', 'insert', 'items', 'iter', 'iterfind', 'itertext', 'keys', 'makeelement', 'remove', 'set', 'tag', 'tail', 'text']


In [38]:
tree.write('processed.svg')

In [35]:
# dir(node)
# node.attrib
# node.set('id', 'banana')
node.get('id')

'banana'

In [19]:
for x in root.iter('{http://www.w3.org/2000/svg}g'):
    print(x)

<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3CFA60>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3CFC90>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3A9440>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3A94E0>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3A97B0>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3A9C10>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3A9EE0>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3A9FD0>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3AA390>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3AA6B0>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3AA930>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3AAAC0>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3AAC50>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3AADE0>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3AAED0>
<Element '{http://www.w3.org/2000/svg}g'

In [20]:
for x in root.findall('svg:g', ns):
    print(x)

<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3CFA60>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3CFC90>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3A9440>
<Element '{http://www.w3.org/2000/svg}g' at 0x000002821E3AB560>
